In [1]:
import numpy as np
import matplotlib.pyplot as plt
import Circuit
from EC08 import EC08
import pandas as pd
from numpy import random

In [2]:
frequencies = Circuit.dec_step(1e-2, 1e5, 72)

In [3]:
import random
# from openpyxl import Workbook
samples=50000
# Generate random values for three variables
Rs = [random.uniform(15,35) for _ in range(samples)]
R1 = [random.uniform(15,35) for _ in range(samples)]
R2 = [random.uniform(15,35) for _ in range(samples)]
CPE_Y1 = [random.uniform(1e-6, 1e-3) for _ in range(samples)]
CPE_n1 = [random.uniform(0.5, 1) for _ in range(samples)]
CPE_Y2 = [random.uniform(1e-6, 1e-3) for _ in range(samples)]
CPE_n2 = [random.uniform(0.5, 1) for _ in range(samples)]


In [4]:
output_array_8 = np.stack((Rs,R1,R2, CPE_Y1, CPE_n1,CPE_Y2, CPE_n2), axis=1)

In [5]:
num_rows = len(output_array_8)
num_rows

50000

In [6]:
num_freq = len(output_array_8)
num_rows = 72
num_columns = 3

# Create an empty numpy array to store the frequency response data
frequency_response_array_8 = np.empty((num_freq, num_rows, num_columns))
# len(frequency_response_array[0][0])

In [7]:
# As output_array is a 2D NumPy array
for idx in range(output_array_8.shape[0]):
    row = output_array_8[idx]
    b = EC08(*row)
    Z = b.freq_response(frequencies)

    # Store the frequency values  in the first column
    frequency_response_array_8[idx, :, 0] = frequencies

    # Store the Absolute Value of Z in the second column
    frequency_response_array_8[idx, :, 1] = np.abs(Z)

    # Store the Phase values of Z in the third column
    frequency_response_array_8[idx, :, 2] = np.angle(Z)


# Now, frequency_response_array has the shape (50000, 72, 3)



In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Assuming frequency_response_array has shape (50000, 72, 3) and output_array has shape (50000, 3)

# Reshape frequency_response_array to have shape (50000, 216)
X = frequency_response_array_8.reshape((frequency_response_array_8.shape[0], -1))

y = output_array_8

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("Training R^2 Score:", train_score)
print("Testing R^2 Score:", test_score)


Training R^2 Score: 0.532262838171785
Testing R^2 Score: 0.5268152209929051


In [10]:
for i in range(1000):    
    random_idx = np.random.randint(0, len(frequency_response_array_8))
    # Select the random element from frequency_response_array
    random_element = frequency_response_array_8[random_idx]

    # Reshape the random element to match the input shape expected by the model
    reshaped_element = random_element.reshape(1, -1)

    # Predict the corresponding output value
    predicted_output = model.predict(reshaped_element)

    # Extract the corresponding actual output value from the output array
    actual_output = output_array_8[random_idx]

    # Print the predicted and actual output values for comparison
    print("Predicted Output:",predicted_output)
    print("Actual Output:",actual_output)


Predicted Output: [[3.45820312e+01 1.70000000e+01 1.80000000e+01 2.74658203e-04
  7.03125000e-01 3.05175781e-04 6.71875000e-01]]
Actual Output: [3.49810359e+01 1.63683115e+01 1.71395898e+01 1.50320854e-04
 5.98457911e-01 1.37658094e-04 6.95554900e-01]
Predicted Output: [[3.17128906e+01 2.70000000e+01 2.80000000e+01 8.54492188e-04
  8.28125000e-01 9.00268555e-04 8.12500000e-01]]
Actual Output: [3.18627773e+01 2.60367466e+01 2.75320596e+01 9.51470852e-04
 9.05537512e-01 9.91982001e-04 8.35976150e-01]
Predicted Output: [[2.34824219e+01 2.80000000e+01 2.90000000e+01 6.71386719e-04
  8.90625000e-01 6.86645508e-04 8.90625000e-01]]
Actual Output: [2.33670893e+01 2.62824667e+01 2.93997760e+01 5.38593851e-04
 8.98830596e-01 7.52474811e-04 8.71517061e-01]
Predicted Output: [[2.74765625e+01 1.90000000e+01 2.00000000e+01 7.93457031e-04
  7.34375000e-01 7.78198242e-04 7.34375000e-01]]
Actual Output: [2.74330528e+01 1.97346034e+01 1.88578492e+01 9.45288918e-04
 5.42955686e-01 9.31320428e-04 9.868611

In [11]:
from sklearn.metrics import mean_squared_error,accuracy_score

# Predict outputs for the entire test set
predicted_outputs = model.predict(X_test)

# Compute mean squared error
mse = mean_squared_error(y_test, predicted_outputs)
accuracy=100-mse
# Print the mean squared error
print("Mean Squared Error:", mse)
print("Accuracy:",accuracy)


Mean Squared Error: 4.9514249919795
Accuracy: 95.0485750080205


In [12]:
np.save('frequency_response_array_8.npy',frequency_response_array_8)